In [23]:
import pickle
import torch
import numpy
from torch_geometric.loader import HGTLoader
from torch_geometric.nn import Linear, HeteroConv, GATConv
import gc

In [24]:
with open('./data/okved_graph_without_dgl.pickle', 'rb') as file:
    nodes = pickle.load(file)
    u = pickle.load(file)
    v = pickle.load(file)
    edata = pickle.load(file)
    ndata = pickle.load(file)

uv = torch.zeros(2, u.shape[0], dtype=torch.int64)
uv[0, :] = u
uv[1, :] = v

In [25]:
from torch_geometric.data import HeteroData

data = HeteroData()
# создаем узлы как сразу матрицы, в которую вложена информация об узлах
data['code'].x = ndata['feat'].type(torch.float32)

# создаем связи трех типов
data['code', 'type_0', 'code'].edge_index = uv[:, edata['type'] == 0]
data['code', 'type_1', 'code'].edge_index = uv[:, edata['type'] == 1]
data['code', 'type_2', 'code'].edge_index = uv[:, edata['type'] == 2]

# добавляем атрибуты на связи
data['code', 'type_0', 'code'].edge_attr = edata['norm'][edata['type'] == 0].type(torch.float32).reshape(-1, 1)
data['code', 'type_1', 'code'].edge_attr = edata['norm'][edata['type'] == 1].type(torch.float32).reshape(-1, 1)
data['code', 'type_2', 'code'].edge_attr = edata['norm'][edata['type'] == 2].type(torch.float32).reshape(-1, 1)

In [26]:
class NegativeSampler:

    def __init__(self, graph, batch_size=1):

        negative_pairs = torch.zeros(2, graph.num_nodes ** 2 - graph.num_edges, dtype=torch.int64)
        nums_uniq = [graph[type_]['edge_index'].shape[1] for type_ in graph.edge_types]
        self.sizes = [int(numpy.ceil(ui / sum(nums_uniq) * batch_size * 32)) for ui in nums_uniq]

        j = 0
        for k in range(graph.num_nodes):
            u_pos, v_pos = uv[:, uv[0] == k]
            v_neg = torch.isin(torch.arange(graph.num_nodes), v_pos, invert=True)
            v_neg = torch.arange(graph.num_nodes, dtype=torch.int64)[v_neg]
            try:
                negative_pairs[0, j:j + v_neg.shape[0]] = torch.tile(u_pos[0], v_neg.shape)
                negative_pairs[1, j:j + v_neg.shape[0]] = v_neg
            except RuntimeError:
                negative_pairs = negative_pairs[:, :j]
                break
            j += v_neg.shape[0]

        self.negative_pairs = negative_pairs
        self.negative_data = HeteroData()
        self.negative_data['code'].x = graph['code'].x

    def __call__(self):
        negative_data = self.negative_data.clone()
        negative_pairs = self.negative_pairs
        negative_data['code', 'type_0', 'code'].edge_index = negative_pairs[:, torch.randint(negative_pairs.shape[1],
                                                                                             (self.sizes[0],))]
        negative_data['code', 'type_1', 'code'].edge_index = negative_pairs[:, torch.randint(negative_pairs.shape[1],
                                                                                             (self.sizes[1],))]
        negative_data['code', 'type_2', 'code'].edge_index = negative_pairs[:, torch.randint(negative_pairs.shape[1],
                                                                                             (self.sizes[2],))]

        # добавляем атрибуты на связи
        negative_data['code', 'type_0', 'code'].edge_attr = torch.randn(self.sizes[0], 1)
        negative_data['code', 'type_1', 'code'].edge_attr = torch.randn(self.sizes[1], 1)
        negative_data['code', 'type_2', 'code'].edge_attr = torch.randn(self.sizes[2], 1)

        return negative_data

In [27]:
BATCH_SIZE = 2637  # При создании dataloader берется BATCH_SIZE узлов

negative_sampler = NegativeSampler(data, BATCH_SIZE)
negative_sampler()

HeteroData(
  code={ x=[2637, 312] },
  (code, type_0, code)={
    edge_index=[2, 33826],
    edge_attr=[33826, 1]
  },
  (code, type_1, code)={
    edge_index=[2, 1015],
    edge_attr=[1015, 1]
  },
  (code, type_2, code)={
    edge_index=[2, 49545],
    edge_attr=[49545, 1]
  }
)

In [28]:
# если на маке - mps, если cuda - то cuda, иначе cpu
device = 'cpu'  # mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cpu'

In [29]:
data = data.to(device)

positive_loader = HGTLoader(
    data,
    # Для каждого типа связи, которых BATCH_SIZE штук, мы генерируем 128 соседей по 2 раза
    num_samples={key: [128] * 2 for key in data.node_types},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=BATCH_SIZE,
    input_nodes=('code', torch.arange(data.num_nodes)),
)

In [30]:
class ThreeLayers(torch.nn.Module):
    """
    Модель, которая возвращает при вызове метода forward уверенность в том, что такая связь существует

    Methods
    -------
    forward(graph) - делает проход вперед, получая эмебеддинги узлов
    predict(graph) - на эмбеддингах строит прогноз, о том сущесвует ли связь
    """

    def __init__(self, hidden_channels, num_layers=1, emb_size=256):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(
            HeteroConv({
                ('code', 'type_0', 'code'): GATConv((-1, -1), hidden_channels),
                ('code', 'type_1', 'code'): GATConv((-1, -1), hidden_channels),
                ('code', 'type_2', 'code'): GATConv((-1, -1), hidden_channels),
            }, aggr='mean'))
        for _ in range(num_layers - 1):
            conv = HeteroConv({
                ('code', 'type_0', 'code'): GATConv((-1, -1), hidden_channels),
                ('code', 'type_1', 'code'): GATConv((-1, -1), hidden_channels),
                ('code', 'type_2', 'code'): GATConv((-1, -1), hidden_channels),
            }, aggr='mean')
            self.convs.append(conv)

        self.convs.append(
            HeteroConv({
                ('code', 'type_0', 'code'): GATConv((-1, -1), emb_size),
                ('code', 'type_1', 'code'): GATConv((-1, -1), emb_size),
                ('code', 'type_2', 'code'): GATConv((-1, -1), emb_size),
            }, aggr='mean'))

        self.lin = Linear(emb_size, 1)  # Полносвязный слой для предсказания существования связи


    def forward(self, graph: HeteroData):

        x = graph.x_dict
        for conv in self.convs:
            # [print(i.shape, end='\t') for i in (x, edge_indexes, node_types, edge_types, edge_attr)]
            # [print(i.dtype, end='\t') for i in (x, edge_indexes, node_types, edge_types, edge_attr)]

            x = conv(x, graph.edge_index_dict)

        return x

    def predict(self, graph: HeteroData):
        x = self.lin(self.forward(graph)['code'])
        return torch.sigmoid(x)


model = ThreeLayers(hidden_channels=256, emb_size=256, num_layers=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()


def train(positive, negative):
    positive = positive.clone()
    negative = negative.clone()
    model.train()
    optimizer.zero_grad()
    out = model.predict(positive)
    neg_out = model.predict(negative)
    pred = torch.concat([out, neg_out]).flatten()
    true = torch.zeros(pred.shape[0])
    true[:out.shape[0]] = 1
    true[out.shape[0]:] = 0
    _loss = criterion(pred, true)
    _loss.backward()
    optimizer.step()
    return _loss.item()

In [31]:
with torch.no_grad():  # Проверим, что все окей
    pos = next(iter(positive_loader))
    pos = model.predict(pos)
    neg = model.predict(negative_sampler())

pos.shape, neg.shape

(torch.Size([2637, 1]), torch.Size([2637, 1]))

In [32]:
EPOCHS = 7
neg = negative_sampler()
for epoch in range(EPOCHS):
    losses = []

    for pos in positive_loader:
        losses.append(train(pos, neg))

    print(f'epoch: {epoch + 1:2d} | loss: {numpy.mean(losses): .4f}')
    gc.collect()

epoch:  1 | loss:  0.6934
epoch:  2 | loss:  0.6916
epoch:  3 | loss:  0.6890
epoch:  4 | loss:  0.6840
epoch:  5 | loss:  0.6752
epoch:  6 | loss:  0.6609
epoch:  7 | loss:  0.6389


In [33]:
with torch.no_grad():
    embeddings_model = model(data)['code']

In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold
from tqdm import tqdm
import pandas as pd

embeddings_bert = numpy.load('./data/okved_embeddings.npy')
okved_consumption = pd.read_csv('./data/okved_consumption.csv')
okved_data = pd.read_csv('./data/okved_2014_w_sections.csv')

In [35]:
embeddings_model_2d = TSNE(n_components=2, init='random', learning_rate=200).fit_transform(embeddings_model)
embeddings_bert_2d = TSNE(n_components=2, init='random', learning_rate=200).fit_transform(embeddings_bert)

In [36]:
import plotly.express as px
import plotly

plotly.offline.init_notebook_mode(connected=True)
sections = list(map(str, okved_data['section_id'].values))

fig = px.scatter(x=embeddings_model_2d[1:, 0], y=embeddings_model_2d[1:, 1], color=sections,
                 title='<b>TSNE embeddings. OUR MODEL</b>', hover_name=okved_data['name_section'])

fig

In [44]:
fig = px.scatter(x=embeddings_bert_2d[:, 0], y=embeddings_bert_2d[:, 1], color=sections,
                 title='<b>TSNE embeddings. BERT</b>', hover_name=okved_data['name_section'])

fig

In [45]:
from help.preprocessing_okved import fix_okved

id_to_code = okved_data['native_code'].to_dict()
id_to_code[0] = '0'
code_to_id = {v: u for u, v in id_to_code.items()}

prov_indices = okved_consumption['okved_provider'].map(fix_okved).map(code_to_id)
cons_indices = okved_consumption['okved_consumer'].map(fix_okved).map(code_to_id)

X_bert = numpy.column_stack((embeddings_bert[prov_indices], embeddings_bert[cons_indices]))
X_bert = StandardScaler().fit_transform(X_bert)

X_model = numpy.column_stack((embeddings_model[prov_indices], embeddings_model[cons_indices]))
X_model = StandardScaler().fit_transform(X_model)

y = okved_consumption['normalized_consumption']

## Линейная модель

In [46]:
model_linear = LinearRegression().fit(X_model, y)
score = model_linear.score(X_model, y)
print(f'Модель: {score: 0.4f}')

model_linear = LinearRegression().fit(X_bert, y)
score = model_linear.score(X_bert, y)
print(f'Берт: {score: 0.4f}')

Модель:  0.1001
Берт:  0.0021


## Перцептрон

In [47]:
from models.mlp_regressor import MLPRegressorTorch

X_model_th = torch.FloatTensor(X_model).to(device)
X_bert_th = torch.FloatTensor(X_bert).to(device)
y_th = torch.FloatTensor(y.values).to(device)

In [48]:
kf = KFold(n_splits=5)

model_nn = MLPRegressorTorch(X_model.shape[1], (128,)).to(device)
val_scores = []
for i, (train, val) in enumerate(kf.split(X_model_th)):
    model_nn.fit(X_model_th[train], y_th[train], val=[X_model_th[val], y_th[val]], epochs=100 * (11 - 2 * i))
    val_scores.append(model_nn.r2_score(X_model_th[val], y_th[val]))

print(f'AVG CROSS VAL model R2: {numpy.mean(val_scores): 0.4f}')
print(f'AVG model R2: {model_nn.r2_score(X_model_th, y_th): 0.4f}')

AVG CROSS VAL model R2: -0.1873
AVG model R2:  0.3374


In [49]:
kf = KFold(n_splits=5)

model_nn = MLPRegressorTorch(X_bert.shape[1], (128,)).to(device)
val_scores = []
for i, (train, val) in enumerate(kf.split(X_bert_th)):
    model_nn.fit(X_bert_th[train], y_th[train], val=[X_bert_th[val], y_th[val]], epochs=100 * (11 - 2 * i))
    val_scores.append(model_nn.r2_score(X_bert_th[val], y_th[val]))

print(f'AVG CROSS VAL model R2: {numpy.mean(val_scores): 0.4f}')
print(f'AVG model R2: {model_nn.r2_score(X_bert_th, y_th): 0.4f}')

AVG CROSS VAL model R2:  0.3835
AVG model R2:  0.9907


In [50]:
bert_scores = []
model_scores = []

bert_scores_all = []
model_scores_all = []

kf = KFold(n_splits=5)

for j in tqdm(range(10)):

    model_nn = MLPRegressorTorch(X_model.shape[1], (128,)).to(device)  # эмбеддинги нашей моделью
    val_scores = []
    # Обучаем модель с помощью кросс валидации
    for i, (train, val) in enumerate(kf.split(X_model_th)):
        # Во время обучения контролируем переобучение с помощью валидационных данных
        model_nn.fit(X_model_th[train], y_th[train],
                     val=[X_model_th[val], y_th[val]],
                     epochs=100 * (11 - 2 * i))
        val_scores.append(model_nn.r2_score(X_model_th[val], y_th[val]))

    model_scores.append(numpy.mean(val_scores))
    model_scores_all.append(model_nn.r2_score(X_model_th, y_th))
    model_nn = MLPRegressorTorch(X_bert.shape[1], (128,)).to(device)

    val_scores = []

    for i, (train, val) in enumerate(kf.split(X_model_th)):
        model_nn.fit(X_bert_th[train], y_th[train],
                     val=[X_bert_th[val], y_th[val]],
                     epochs=100 * (11 - 2 * i))

        val_scores.append(model_nn.r2_score(X_bert_th[val], y_th[val]))

    bert_scores.append(numpy.mean(val_scores))
    bert_scores_all.append(model_nn.r2_score(X_bert_th, y_th))

100%|██████████| 10/10 [06:25<00:00, 38.55s/it]


In [52]:
print(f'Модель: {numpy.mean(model_scores_all):.2f} ± {numpy.std(model_scores_all):.2f}')
print(f'Берт: {numpy.mean(bert_scores_all):.2f} ± {numpy.std(bert_scores_all):.2f}')

Модель: 0.32 ± 0.02
Берт: 0.99 ± 0.00
